In [40]:
import pandas as pd
import random
import seaborn as sns
from scipy import stats
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler,OrdinalEncoder
import pandas as pd


Import de la base de données

In [79]:
# Définir le nombre de lignes à lire
nrows = 1000000

# Lire le fichier entier
df = pd.read_csv("raw_data/valeursfoncieres-2023.txt", sep="|", dtype=str)

# Sélectionner 100,000 lignes au hasard
chunk = df.sample(n=nrows, random_state=1)

# Liste des colonnes d'intérêt
cols = ['Date mutation', 'Nature mutation', 'Valeur fonciere',
        'Type de voie', 'Code postal', 'Surface Carrez du 1er lot',
        'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot',
        'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot',
        'Nombre de lots', 'Code type local', 'Surface reelle bati',
        'Nombre pieces principales', 'Surface terrain']

departements = ['75', '13', '69', '31', '06', '44', '34', '67', '33', '59']

# Sélectionner les colonnes
chunk = chunk[cols]
chunk = chunk[chunk['Code postal'].astype(str).str.startswith(tuple(departements))]

Cleaning

In [80]:
# Suppression des lignes qui n'ont pas de 'Valeur fonciere' ou de 'Code postal'
chunk.dropna(subset=['Valeur fonciere', 'Code postal'], inplace=True)

# Filtrer pour ne garder que les 'Vente' classiques
chunk = chunk[chunk['Nature mutation'] == 'Vente']
chunk = chunk[(chunk['Code type local'] == '1') | (chunk['Code type local'] == '2')]
chunk['Date mutation'] = pd.to_datetime(chunk['Date mutation'], format='%d/%m/%Y')

# Colonnes Carrez à traiter
carrez_cols = ['Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot',
               'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot',
               'Surface Carrez du 5eme lot']

# Remplacer les virgules par des points dans les colonnes numériques
chunk['Valeur fonciere'] = chunk['Valeur fonciere'].str.replace(',', '.').astype(float)
chunk[carrez_cols] = chunk[carrez_cols].apply(lambda col: col.str.replace(',', '.').astype(float))

# Remplacer les NaN dans les colonnes Carrez par 0
chunk[carrez_cols] = chunk[carrez_cols].fillna(0)
chunk['somme surface carrez'] = chunk[carrez_cols].sum(axis=1)

# Supprimer les colonnes contenant les surfaces carrez
chunk.drop(columns=carrez_cols, inplace=True)
chunk[['Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']] = chunk[['Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']].fillna(0).astype(float)
chunk[['Nombre de lots', 'Code type local']] = chunk[['Nombre de lots', 'Code type local']].fillna(0).astype(int)

# Filtrer pour enlever les lignes où 'Nombre de lots', 'Surface reelle bati', ou 'somme surface carrez' sont égales à 0
chunk = chunk[(chunk['Nombre de lots'] != 0) |
              (chunk['Surface reelle bati'] != 0) |
              (chunk['somme surface carrez'] != 0)]

# Ajouter un zéro à la fin des codes postaux qui ont moins de 5 caractères
chunk['Code postal'] = chunk['Code postal'].astype(str).apply(lambda x: x + '0' * (5 - len(x)) if len(x) < 5 else x)

# Clean Code Type Local : d'int à object
chunk['Code type local'] = chunk['Code type local'].astype('object')

# Clean Nombre pieces principales : de float à int
chunk['Nombre pieces principales'] = chunk['Nombre pieces principales'].astype(int)

# Ajout d'une colonne mois, d'une colonne annéee, suppression de Date Mutation et Nature Mutation
chunk['Month mutation'] = chunk['Date mutation'].dt.month
chunk['Year mutation'] = chunk['Date mutation'].dt.year
chunk = chunk.drop(columns=['Date mutation', 'Nature mutation'])

In [81]:
# Clean du nom des colonnes
def clean_column_names(df):
    # Remplacer les majuscules par des minuscules et les espaces par des underscores
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

chunk = clean_column_names(chunk)

In [82]:
# Gestion des outliers

# Calcul du Z-score
df=chunk.copy()
df['z_score'] = stats.zscore(df['valeur_fonciere'])
outliers = df[df['z_score'].abs() > 3]  # Seuil de 3

#Remove outliers
chunk = df[df['z_score'].abs() <= 3]

#Drop z_score
chunk = chunk.drop(['z_score'], axis=1)

Preprocessing

In [86]:
# Définir les pipelines de prétraitement
preproc_robust = make_pipeline(RobustScaler())
preproc_standard = make_pipeline(StandardScaler())
preproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preproc_ordinal = make_pipeline(OrdinalEncoder())

# Définir le transformer de colonnes
preproc_baseline = make_column_transformer(
    (preproc_standard, ['surface_reelle_bati']),
    (preproc_robust, ['surface_terrain', 'somme_surface_carrez','valeur_fonciere']),
    (preproc_categorical_baseline, ['code_type_local', 'type_de_voie', 'code_postal']),
    (preproc_ordinal, ['nombre_pieces_principales', 'month_mutation', 'year_mutation']),
    remainder='passthrough')

# Préparer les données
X = chunk.copy()
y = chunk.valeur_fonciere.copy()

# Transformer les données
X_trans = preproc_baseline.fit_transform(X)

# Fonction pour obtenir les noms des colonnes
def get_feature_names(column_transformer):
    feature_names = []
    for name, transformer, columns in column_transformer.transformers_:
        if hasattr(transformer, 'get_feature_names_out'):
            # Obtenir les noms de fonctionnalités pour chaque transformeur
            feature_names.extend(transformer.get_feature_names_out())
        else:
            feature_names.extend(columns)
    return feature_names

# Obtenir les noms des colonnes
feature_names = get_feature_names(preproc_baseline)

# Convertir en DataFrame si la sortie est une matrice creuse
if hasattr(X_trans, "toarray"):
    X_trans_dense = X_trans.toarray()
else:
    X_trans_dense = X_trans
    
# Créer la DataFrame avec les noms de colonnes
X_trans_df = pd.DataFrame(X_trans_dense, columns=feature_names)

In [88]:
correlation_matrix = X_trans_df.corr()
X_corr=X_trans_df.drop(columns='valeur_fonciere').copy()
X_corr=X_corr.corr()
#column_names = correlation_matrix.columns
#sns.heatmap(correlation_matrix, xticklabels=column_names, yticklabels=column_names,cmap= "bwr")
#print(correlation_matrix['valeur_fonciere'].abs().sort_values(ascending=False))
print(X_corr)

                           surface_reelle_bati  surface_terrain  \
surface_reelle_bati                   1.000000         0.122906   
surface_terrain                       0.122906         1.000000   
somme_surface_carrez                 -0.019184        -0.050423   
code_type_local_1                     0.518280         0.075675   
code_type_local_2                    -0.518280        -0.075675   
...                                        ...              ...   
code_postal_75900                     0.000764        -0.000209   
nombre_pieces_principales             0.829500         0.090316   
month_mutation                        0.002001         0.052674   
year_mutation                              NaN              NaN   
nombre_de_lots                       -0.289780        -0.114125   

                           somme_surface_carrez  code_type_local_1  \
surface_reelle_bati                   -0.019184           0.518280   
surface_terrain                       -0.050423        